In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### 问题：
1.第三张表格的处理

2.训练集和测试集

3.样本极不平衡问题：负样本欠采样——需要对全部数据完成特征提取之后再进行采样

4.如何进行特征工程


In [25]:
# 读取客户文件
cust_dataset = pd.read_csv('../cust_dataset.csv')
cust_dataset = pd.DataFrame(cust_dataset)

# 读取事件文件
event_dataset = pd.read_csv('../event_dataset.csv')
event_dataset = pd.DataFrame(event_dataset)
print(cust_dataset.shape)
print(event_dataset.shape)
# print(cust_dataset.head())
# print(event_dataset.head())

# 读取产品文件
excel_file = pd.ExcelFile('../productLabels_multiSpreadsheets.xlsx', engine='openpyxl') 
all_sheets = pd.read_excel(excel_file, sheet_name=None) 
# print("工作表名称：", list(all_sheets.keys()))
D_dataset = pd.DataFrame(all_sheets['D'])
D_dataset.columns = [col if col == 'prod_id' else f'D_{col}' for col in D_dataset.columns]

C_dataset = pd.DataFrame(all_sheets['C'])
C_dataset.columns = [col if col == 'prod_id' else f'C_{col}' for col in C_dataset.columns]

A_dataset = pd.DataFrame(all_sheets['A'])
A_dataset.columns = [col if col == 'prod_id' else f'A_{col}' for col in A_dataset.columns]

N_dataset = pd.DataFrame(all_sheets['N'])
N_dataset.columns = [col if col == 'prod_id' else f'N_{col}' for col in N_dataset.columns]

P_dataset = pd.DataFrame(all_sheets['P'])
P_dataset.columns = [col if col == 'prod_id' else f'P_{col}' for col in P_dataset.columns]


(1108827, 7)
(338939, 9)


In [31]:
cust_dataset.head()
# print(len(cust_dataset['cust_no'].unique()))
# print(len(cust_dataset['loc_cd'].unique()))
# print(cust_dataset['edu_bg'].unique())
# print(cust_dataset['marriage_situ_cd'].unique())

# print(len(event_dataset['cust_no'].unique()))
# print(event_dataset['prod_id'].unique())
# print(event_dataset['event_id'].unique())
# print(event_dataset['event_type'].unique())
# print(event_dataset['event_level'].unique())

# event_dataset['event_type'].value_counts()

,cust_no,birth_ym,loc_cd,gender,init_dt,edu_bg,marriage_situ_cd
0,CDB91DCCE198B10A522FE2AABF6A8D81,1942-03,L001,M,2011-03-01,NaN,NaN
1,9307AC85C179D8E388DC776DB6283534,1986-08,L001,F,2010-12-14,NaN,NaN
2,9FA3282573CEB37A5E9BC1C38088087F,1950-11,L001,M,2010-07-24,NaN,NaN
3,CB0D6827A924C7FFDD9DD57BF5CE9358,1951-08,L001,F,2011-09-16,NaN,NaN
4,797E3448CF516A52ADBE6DB33626B50E,1957-05,L001,M,2011-10-17,NaN,NaN


20251019 陈实

cust_dataset只是罗列了全部的用户信息，但没有反映事件类型，所以将event_dataset与cust_dateset左连接，增加事件类型信息,得到cust_event

发现：cust_event的cust_no 与 cust_dataset的cust_no 一致，均为133853个

In [ ]:
# 数据连接：我们以事件表为核心，增加了客户、产品的信息
event_cust = pd.merge(event_dataset,cust_dataset, on='cust_no', how = 'left')
event_cust = pd.merge(event_cust,D_dataset,on = 'prod_id',how = 'left')
event_cust = pd.merge(event_cust,C_dataset,on = 'prod_id',how = 'left')
event_cust = pd.merge(event_cust,N_dataset,on = 'prod_id',how = 'left')
event_cust = pd.merge(event_cust,A_dataset,on = 'prod_id',how = 'left')
event_cust = pd.merge(event_cust,P_dataset,on = 'prod_id',how = 'left')

In [29]:
print(event_cust.shape)
print(len(event_cust['cust_no'].unique()))
event_cust.head()

(338939, 32)
133953


,cust_no,prod_id,event_id,event_type,event_level,event_date,event_term,event_rate,event_amt,birth_ym,...,C_new_flag,N_channel_type,N_channel_type2,A_frtn_type,A_fr_period_type,A_fr_prod_attr,A_fr_prod_type,A_fr_risk_level,P_pay_type1,P_pay_type2
0,B7CC9BA112B6CDA18A8DC1D360A472C7,C0001,E0001,B,B,2024-05-15,60.0,NaN,NaN,1978-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,83294DBFC428E8594798CE0FAB0DADE3,C0001,E0001,B,B,2024-05-06,60.0,NaN,NaN,1994-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,374E6347DD854EE328B92BD429281459,C0001,E0001,B,B,2024-06-24,60.0,NaN,NaN,1966-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0A76A65A0975BC5BC880A354D9C3559F,C0001,E0001,B,B,2024-03-18,60.0,NaN,NaN,1988-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,148C839CDEAFA9971B634DC76B5A2C60,C0001,E0001,B,B,2024-08-09,60.0,NaN,NaN,1989-04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# 特征工程
# event_cust.groupby(['event_type'])['event_type'].count()
event_cust.columns

Index(['cust_no', 'prod_id', 'event_id', 'event_type', 'event_level',
       'event_date', 'event_term', 'event_rate', 'event_amt', 'birth_ym',
       'loc_cd', 'gender', 'init_dt', 'edu_bg', 'marriage_situ_cd',
       'D_interval_level', 'D_deposit_type1', 'D_deposit_type2',
       'C_credit_level', 'C_credit_amt_cd', 'C_credit_type1', 'C_credit_type2',
       'C_new_flag', 'N_channel_type', 'N_channel_type2', 'A_frtn_type',
       'A_fr_period_type', 'A_fr_prod_attr', 'A_fr_prod_type',
       'A_fr_risk_level', 'P_pay_type1', 'P_pay_type2'],
      dtype='object')